In [77]:
import numpy as np
import time
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader

## Рекуррентные нейросети 2

In [78]:
def generate_sequences(num_samples, seq_length):
    data = []
    for _ in range(num_samples):
        x = np.random.randint(0, 10, size=seq_length)  
        y = [x[0]]  
        y += [(x[i] + x[0]) % 10 for i in range(1, seq_length)]  
        data.append((x, y))
    return data

In [79]:
num_samples = 10000
seq_length = 10
data = generate_sequences(num_samples, seq_length)

train_data = data[:8000]
test_data = data[8000:]

In [80]:
class SequenceDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        x, y = self.data[idx]
        return torch.tensor(x, dtype=torch.long), torch.tensor(y, dtype=torch.long)

In [81]:
train_dataset = SequenceDataset(train_data)
test_dataset = SequenceDataset(test_data)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

### RNN

In [82]:
class SimpleRNN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleRNN, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)  
        self.rnn = nn.RNN(hidden_dim, hidden_dim, batch_first=True)  
        self.fc = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        x = self.embedding(x)  
        out, _ = self.rnn(x)  
        out = self.fc(out)  
        return out

In [83]:
input_dim = 10  
hidden_dim = 64 
output_dim = 10  

In [84]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = SimpleRNN(input_dim, hidden_dim, output_dim).to(device)

In [85]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [86]:
def train_model(model, train_loader, test_loader, criterion, optimizer, num_epochs, device):
    for epoch in range(num_epochs):
        start_time = time.time()
        
        model.train()
        train_loss = 0
        train_correct = 0
        train_total = 0

        for encrypted_seq, original_seq in train_loader:
            encrypted_seq = encrypted_seq.to(device)
            original_seq = original_seq.to(device)

            output = model(encrypted_seq)
            
            output = output.view(-1, output.size(-1))  
            original_seq = original_seq.view(-1)  

            loss = criterion(output, original_seq)
            train_loss += loss.item()

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        model.eval()
        test_loss = 0
        test_correct = 0
        test_total = 0

        with torch.no_grad():
            for encrypted_seq, original_seq in test_loader:
                encrypted_seq = encrypted_seq.to(device)
                original_seq = original_seq.to(device)

                output = model(encrypted_seq)

                output = output.view(-1, output.size(-1))
                original_seq = original_seq.view(-1)

                loss = criterion(output, original_seq)
                test_loss += loss.item()

        epoch_time = time.time() - start_time

        print(f"Ep [{epoch+1}/{num_epochs}], Time: {epoch_time:.2f} sec, Train Loss: {train_loss/len(train_loader):.4f}, Test Loss: {test_loss/len(test_loader):.4f}")

In [87]:
train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=10,
    device=device
)

Ep [1/10], Time: 0.78 sec, Train Loss: 2.2184, Test Loss: 2.0397
Ep [2/10], Time: 0.59 sec, Train Loss: 1.9304, Test Loss: 1.8489
Ep [3/10], Time: 0.56 sec, Train Loss: 1.8242, Test Loss: 1.7824
Ep [4/10], Time: 0.58 sec, Train Loss: 1.7630, Test Loss: 1.7202
Ep [5/10], Time: 0.57 sec, Train Loss: 1.6830, Test Loss: 1.5530
Ep [6/10], Time: 0.57 sec, Train Loss: 1.2098, Test Loss: 0.8805
Ep [7/10], Time: 0.60 sec, Train Loss: 0.6816, Test Loss: 0.4638
Ep [8/10], Time: 0.61 sec, Train Loss: 0.3289, Test Loss: 0.2213
Ep [9/10], Time: 0.61 sec, Train Loss: 0.1848, Test Loss: 0.1345
Ep [10/10], Time: 0.63 sec, Train Loss: 0.1096, Test Loss: 0.0877


In [88]:
def predict_sequence(model, sequence, device):
    model.eval()
    with torch.no_grad():
        sequence = torch.tensor(sequence, dtype=torch.long).unsqueeze(0).to(device)
        output = model(sequence)
        predictions = torch.argmax(output, dim=2).squeeze(0).tolist()
        return predictions


In [89]:
test_sequence = np.random.randint(0, 10, size=10)  
predicted_output = predict_sequence(model, test_sequence, device)

print(f"Вводная последовательность: {test_sequence}")
print(f"Предсказанная последовательность: {predicted_output}")

Вводная последовательность: [6 6 2 0 8 6 5 6 6 3]
Предсказанная последовательность: [6, 2, 8, 6, 4, 2, 1, 2, 2, 9]


### LSTM

In [90]:
class SimpleLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleLSTM, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)  
        self.lstm = nn.LSTM(hidden_dim, hidden_dim, batch_first=True)  
        self.fc = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        x = self.embedding(x)  
        out, _ = self.lstm(x)  
        out = self.fc(out)  
        return out

In [91]:
model = SimpleLSTM(input_dim, hidden_dim, output_dim).to(device)

In [92]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [93]:
train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=10,
    device=device
)

Ep [1/10], Time: 0.65 sec, Train Loss: 2.0432, Test Loss: 1.1965
Ep [2/10], Time: 0.65 sec, Train Loss: 0.4239, Test Loss: 0.1089
Ep [3/10], Time: 0.67 sec, Train Loss: 0.0604, Test Loss: 0.0344
Ep [4/10], Time: 0.63 sec, Train Loss: 0.0243, Test Loss: 0.0175
Ep [5/10], Time: 0.63 sec, Train Loss: 0.0136, Test Loss: 0.0107
Ep [6/10], Time: 0.64 sec, Train Loss: 0.0087, Test Loss: 0.0072
Ep [7/10], Time: 0.63 sec, Train Loss: 0.0061, Test Loss: 0.0052
Ep [8/10], Time: 0.67 sec, Train Loss: 0.0044, Test Loss: 0.0039
Ep [9/10], Time: 0.67 sec, Train Loss: 0.0034, Test Loss: 0.0030
Ep [10/10], Time: 0.62 sec, Train Loss: 0.0026, Test Loss: 0.0024


In [94]:
test_sequence = np.random.randint(0, 10, size=10)  
predicted_output = predict_sequence(model, test_sequence, device)

print(f"Вводная последовательность: {test_sequence}")
print(f"Предсказанная последовательность: {predicted_output}")

Вводная последовательность: [9 9 7 2 3 0 0 3 6 2]
Предсказанная последовательность: [9, 8, 6, 1, 2, 9, 9, 2, 5, 1]


### GRU

In [95]:
class SimpleGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(SimpleGRU, self).__init__()
        self.embedding = nn.Embedding(input_dim, hidden_dim)  
        self.gru = nn.GRU(hidden_dim, hidden_dim, batch_first=True)  
        self.fc = nn.Linear(hidden_dim, output_dim)  

    def forward(self, x):
        x = self.embedding(x)  
        out, _ = self.gru(x)  
        out = self.fc(out)  
        return out

In [96]:
model = SimpleGRU(input_dim, hidden_dim, output_dim).to(device)

In [97]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [98]:
train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    criterion=criterion,
    optimizer=optimizer,
    num_epochs=10,
    device=device
)

Ep [1/10], Time: 0.64 sec, Train Loss: 2.1926, Test Loss: 1.9463
Ep [2/10], Time: 0.53 sec, Train Loss: 1.0042, Test Loss: 0.2675
Ep [3/10], Time: 0.52 sec, Train Loss: 0.1313, Test Loss: 0.0652
Ep [4/10], Time: 0.56 sec, Train Loss: 0.0442, Test Loss: 0.0307
Ep [5/10], Time: 0.60 sec, Train Loss: 0.0233, Test Loss: 0.0179
Ep [6/10], Time: 0.57 sec, Train Loss: 0.0144, Test Loss: 0.0118
Ep [7/10], Time: 0.56 sec, Train Loss: 0.0098, Test Loss: 0.0084
Ep [8/10], Time: 0.58 sec, Train Loss: 0.0072, Test Loss: 0.0062
Ep [9/10], Time: 0.58 sec, Train Loss: 0.0054, Test Loss: 0.0048
Ep [10/10], Time: 0.62 sec, Train Loss: 0.0042, Test Loss: 0.0038


In [99]:
test_sequence = np.random.randint(0, 10, size=10)  
predicted_output = predict_sequence(model, test_sequence, device)

print(f"Вводная последовательность: {test_sequence}")
print(f"Предсказанная последовательность: {predicted_output}")

Вводная последовательность: [2 8 7 5 2 3 2 1 1 8]
Предсказанная последовательность: [2, 0, 9, 7, 4, 5, 4, 3, 3, 0]
